In [1]:
#from msilib.sequence import InstallUISequence

ModuleNotFoundError: No module named 'msilib'

In [3]:
#pip InstallUISequence --upgrade pydantic

ERROR: unknown command "InstallUISequence"
Note: you may need to restart the kernel to use updated packages.


In [67]:
import pandas as pd
from fastapi import FastAPI, HTTPException, Header
from pydantic import BaseModel, Field
from enum import Enum
import csv
import random
from typing import Optional, List
import html
import csvreview

In [68]:
#import csvreview instead of importt csvreview.py
%run csvreview.py

['BDD' 'Systèmes distribués' 'Streaming de données' 'Docker'
 'Classification' 'Sytèmes distribués' 'Data Science' 'Machine Learning'
 'Automation']
['Test de positionnement' 'Test de validation' 'Total Bootcamp']


In [69]:
#insertion of the main_api.py file that cant be imported in a ipynb file
df=pd.read_csv('https://dst-de.s3.eu-west-3.amazonaws.com/fastapi_fr/questions.csv')
df.head()
list_subject=df.subject.unique()
print(list_subject)
df.subject.value_counts()
list_use=df.use.unique()
print(list_use)
df.use.value_counts()
df.responseD.value_counts()
df.remark.value_counts()

['BDD' 'Systèmes distribués' 'Streaming de données' 'Docker'
 'Classification' 'Sytèmes distribués' 'Data Science' 'Machine Learning'
 'Automation']
['Test de positionnement' 'Test de validation' 'Total Bootcamp']


remark
Gut feelings.     1
Name: count, dtype: int64

In [70]:
app = FastAPI(
    title="API pour la base de questions",
    description="API pour consultation de la base de questions et insertion de nouvelles",
    version="1.0")

In [35]:
#specific to the main.py file
'''
list_use=csvreview.list_use
list_subject=csvreview.list_subject
'''

'\nlist_use=csvreview.list_use\nlist_subject=csvreview.list_subject\n'

In [71]:
# Dictionnaire d'utilisateurs tels que définis dans l'énoncé, mot de passe admin 'escaped'
USERS = {
    "alice":"rabbit",
    "bob":"builder",
    "clementine":"mandarine",
    "admin":"{}".format(html.escape("4dm1N"))
    }

In [72]:
class User(BaseModel):
    user: str
    password: str
    subscription: Optional[str] = None

In [73]:
# Définition d'un énumérateur à partir d'un tableau de données
def define_enum_from_array(array):
  enum = Enum(
      "Enum",
      [(value, value) for value in array]
      )
  return enum

In [74]:
enum_use = define_enum_from_array(list_use)


In [75]:
enum_subject = define_enum_from_array(list_subject)

In [50]:
class New_Question(BaseModel):
    question: str
    #subject: Enum = Field(enum_subject=enum)
    subject: Enum = enum_subject
    use: Enum = enum_use
    correct: Enum = Field(enum_values=['A','B','C','D'])
    response_A: str = Field(max_length=150)
    response_B: str = Field(max_length=150)
    response_C: str = Field(max_length=150)
    response_D: Optional[str] = None
    remark: Optional[str] = None

In [76]:
class Requested_Questions(BaseModel):
    subject: enum_subject
    use: Enum = enum_use
    num_questions: int = Field(description="Number of questions (5, 10, 20)", enum=[5, 10, 20])
#    num_questions: Enum = Field(enum_values=[5,10,20])

In [52]:
def authentification(username: str = Header(None), password: str = Header(None)):
    ''' vérifie que l'utilisateur est enregistré te correctement identifié'''
    if username in USERS and USERS[username] == password:
        return username
    raise HTTPException(status_code=401, detail="Utilisateur ne faisant pas partie des utilisateurs enregistrés, ou identifiants incorrects, merci de contacter l'administrateur")


In [53]:
def is_valid_answer_format(correct_answer: str) -> bool:
    ''' vérifie que la réponse correcte est bien parmi les 4 possibilités.'''
    return correct_answer in ['A', 'B', 'C', 'D']


In [55]:
@app.get('/', name='Bienvenue')
async def get_index():
    '''Bienvenue'''
    return {'greetings': 'Bienvenue'}


In [56]:
@app.get('/users', name='Utilisateurs enregistrés')
async def get_users():
    ''' Retourne la liste des utilisateurs enregistrés'''
    return USERS

In [57]:
@app.get("/check", name= 'vérification fonctionnalité API')
async def check():
    ''' Vérification que l'API est fonctionnelle'''
    return {"message": "l'API est bien fonctionelle"}

In [82]:
async def load_questions_from_csv():
    with open("/root/Coding/FastAPI/Eval/Data/questions.csv", "r") as f:
        reader = csv.DictReader(f, delimiter=",")
        questions = list(reader)
    return questions
    await asyncio.sleep(1)


In [79]:
@app.get("/get_questions/",name='Demande de questions')
async def get_questions(requested_questions:Requested_Questions):
#    questions = get_questions_from_database(requested_question.subject, requested_questions.use, requested_questions.num_questions)
#    questions = await load_questions_from_csv()
    questions = await load_questions_from_csv()
    filtered_questions = [q for q in questions if q['use'] == use and q['subject'] in subject]
    num_requested_questions=min(num_questions, len(filtered_questions))
    selected_questions = random.sample(filtered_questions, num_requested_questions)
    return selected_questions

In [63]:
# API endpoint pour l'ajout de questions par un administrateur
@app.post("/create_question/", name='création de questions par un administrateur')
async def create_question(
    question: New_question, 
    username: str = Header(None), 
    password: str = Header(None)):
    if username != 'admin':
        raise HTTPException(status_code=401, detail="Vous n'êtes pas autorisés")
    if username not in USERS or USERS[username] != password:
        raise HTTPException(status_code=401, detail="Vous n'êtes pas enregistrés ou vos identifiants / mot de passe incorrect(s)")
    if not is_valid_answer_format(question.correct):
        raise HTTPException(status_code=400, detail="Format de la question incorrecte")
    questions = await load_questions_from_csv()
    questions.append(question.dict())
    await save_questions_to_csv(questions)
    return {"message": "Merci, question ajoutée avec succès"}
# Ajout de nouvelles questions à la base de données
async def save_questions_to_csv(questions):
    fieldnames = questions[0].keys()
    async with open("/root/Coding/FastAPI/Eval/Data/questions.csv", "w", newline='') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        csv_writer.writeheader()
        for row in questions:
            csv_writer.writerow(row)

In [83]:

# initiation FastAPI application
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop